In [171]:
# The core scripts for the analysis
using Revise
push!(LOAD_PATH, "../src") 
using DataFrames
using Counterfactuals
using CSV
using GLM
using StatsModels

In [124]:
ENV["COLUMNS"] = 200
w6 = CSV.read("data/w6.csv", DataFrame)
w1 = CSV.read("data/w1.csv", DataFrame)

,PROLIFIC_PID,Gender,Education,Income,Political_Views,Age,Depression,Stress,Anxiety,Fear_COVID,Emotional_Support,Loneliness,slope_new_cases,slope_new_deaths,GatheringStrictness
,String31,Float64,Float64,Float64,Float64,Float64,Missing,Missing,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,5df82e8739bcde5dd044f741,1.0,3.0,1.0,4.0,32.0,missing,missing,42,8.0,16.0,25.0,82.2857,3.28571,1.0
2,5c7e91c1973c990016ea72b0,2.0,5.0,3.0,1.0,55.0,missing,missing,69,24.0,30.0,10.0,82.2857,3.28571,1.0
3,5e67052c757e6149c38ce468,1.0,1.0,1.0,4.0,23.0,missing,missing,57,18.0,14.0,25.0,82.2857,3.28571,1.0
4,5d89106d1719af0001ee23e5,2.0,2.0,1.0,6.0,47.0,missing,missing,27,8.0,40.0,5.0,82.2857,3.28571,1.0
5,5d6394518df2870001a74b42,2.0,2.0,1.0,1.0,23.0,missing,missing,42,23.0,40.0,16.0,82.2857,3.28571,1.0
6,5bdd368719b9ba000157fb12,2.0,3.0,4.0,4.0,28.0,missing,missing,29,10.0,40.0,5.0,82.2857,3.28571,1.0
7,5b68d45687af3100015849a9,2.0,9.0,4.0,1.0,58.0,missing,missing,24,11.0,40.0,6.0,82.2857,3.28571,1.0
8,5a8ce265000dab00018ce084,2.0,4.0,6.0,3.0,50.0,missing,missing,37,6.0,34.0,7.0,82.2857,3.28571,1.0
9,5c72efa770d5a200018028f1,1.0,5.0,2.0,3.0,71.0,missing,missing,27,9.0,39.0,5.0,82.2857,3.28571,1.0


In [117]:
names(data)

21-element Vector{String}:
 "PROLIFIC_PID"
 "Gender"
 "Education"
 "Income"
 "Political_Views"
 "Depression"
 "Stress"
 "Anxiety"
 "Fear_COVID"
 "Emotional_Support"
 ⋮
 "slope_new_deaths"
 "GatheringStrictness"
 "Mandatory_business_closure"
 "Mandatory_PPE_masks"
 "more_than_1_attQ_failed"
 "string_outlier_core"
 "response_consistency"
 "wave"
 "RW1_8"

In [126]:
low_quality = sum.(eachrow(w6[!, ["more_than_1_attQ_failed", "string_outlier_core", "response_consistency"]])) .>= 1

1178-element BitVector:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [197]:
using StatsBase
# syntax: best practice to define a _new_ function
poly(x, n) = x^n

# type of model where syntax applies: here this applies to any model type
const POLY_CONTEXT = Any

# struct for behavior
struct PolyTerm{T,D} <: AbstractTerm
    term::T
    deg::D
end

Base.show(io::IO, p::PolyTerm) = print(io, "poly($(p.term), $(p.deg))")

# for `poly` use at run-time (outside @formula), return a schema-less PolyTerm
poly(t::Symbol, d::Int) = PolyTerm(term(t), term(d))

# for `poly` use inside @formula: create a schemaless PolyTerm and apply_schema
function StatsModels.apply_schema(t::FunctionTerm{typeof(poly)},
                                  sch::StatsModels.Schema,
                                  Mod::Type{<:POLY_CONTEXT})
    apply_schema(PolyTerm(t.args_parsed...), sch, Mod)
end

# apply_schema to internal Terms and check for proper types
function StatsModels.apply_schema(t::PolyTerm,
                                  sch::StatsModels.Schema,
                                  Mod::Type{<:POLY_CONTEXT})
    term = apply_schema(t.term, sch, Mod)
    isa(term, ContinuousTerm) ||
        throw(ArgumentError("PolyTerm only works with continuous terms (got $term)"))
    isa(t.deg, ConstantTerm) ||
        throw(ArgumentError("PolyTerm degree must be a number (got $t.deg)"))
    PolyTerm(term, t.deg.n)
end

function StatsModels.modelcols(p::PolyTerm, d::NamedTuple)
    col = modelcols(p.term, d)
    reduce(hcat, [col.^n for n in 1:p.deg])
end

# the basic terms contained within a PolyTerm (for schema extraction)
StatsModels.terms(p::PolyTerm) = terms(p.term)
# names variables from the data that a PolyTerm relies on
StatsModels.termvars(p::PolyTerm) = StatsModels.termvars(p.term)
# number of columns in the matrix this term produces
StatsModels.width(p::PolyTerm) = p.deg

StatsBase.coefnames(p::PolyTerm) = coefnames(p.term) .* "^" .* string.(1:p.deg)

# output

In [198]:
f = @formula(low_quality ~ 1 + poly(Age, 2) * a)

FormulaTerm
Response:
  y(unknown)
Predictors:
  1
  (b)->poly(b, 2)
  a(unknown)
  (b)->poly(b, 2) & a(unknown)

In [127]:
w6[!, :low_quality] = low_quality

1178-element BitVector:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [158]:
adjustments = ["Income", "Gender", "Age", "Education"]
square_terms = ["Income", "Age", "Education"]
treatment = "low_quality"

"low_quality"

In [135]:
w6[w6[!,"Political_Views"] .=== missing, "Income"] 

Float64[]

In [190]:
@macroexpand @formula(y ~ 1 + age^2)

:(StatsModels.Term(:y) ~ StatsModels.ConstantTerm(1) + StatsModels.capture_call(^, ((age,)->age ^ 2), (:age,), $(Expr(:copyast, :($(QuoteNode(:(age ^ 2)))))), [StatsModels.Term(:age), StatsModels.ConstantTerm(2)]))

In [179]:
fit = glm(Term(:low_quality) ~ Term(:Age) + Term(:Age), w6, Binomial(), ProbitLink())


StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, Matrix{Float64}}

low_quality ~ 1 + Age

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      z  Pr(>|z|)   Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)  -1.85996    0.331697    -5.61    <1e-07  -2.51008    -1.20985
Age          -0.0136971  0.00922284  -1.49    0.1375  -0.0317735   0.00437931
─────────────────────────────────────────────────────────────────────────────

In [189]:
poly(Symbol("Age"), 2)

UndefVarError: UndefVarError: poly not defined

In [185]:
fit = glm(@formula(low_quality ~ Age + Age^2), w6, Binomial(), ProbitLink())


StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, Matrix{Float64}}

low_quality ~ 1 + Age + :(Age ^ 2)

Coefficients:
──────────────────────────────────────────────────────────────────────────────────
                    Coef.   Std. Error      z  Pr(>|z|)    Lower 95%     Upper 95%
──────────────────────────────────────────────────────────────────────────────────
(Intercept)  -2.80747      1.14867      -2.44    0.0145  -5.05881     -0.556124
Age           0.0408984    0.0645889     0.63    0.5266  -0.0856934    0.16749
Age ^ 2      -0.000706804  0.000850672  -0.83    0.4060  -0.00237409   0.000960483
──────────────────────────────────────────────────────────────────────────────────

In [95]:
fit = glm(Term(:low_quality) ~ Term(:Education), w6, Binomial(), ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, Matrix{Float64}}

low_quality ~ 1 + Education

Coefficients:
───────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      z  Pr(>|z|)  Lower 95%   Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)  -2.13642     0.307528   -6.95    <1e-11  -2.73917   -1.53368
Education    -0.0502128   0.0686745  -0.73    0.4647  -0.184812   0.0843868
───────────────────────────────────────────────────────────────────────────

In [93]:
Symbol("low_quality")

:low_quality

In [163]:
(Symbol.(adjustments .* "^2"))

4-element Vector{Symbol}:
 Symbol("Income^2")
 Symbol("Gender^2")
 Symbol("Age^2")
 Symbol("Education^2")

In [164]:
f = Term(Symbol(treatment)) ~ sum(Term.(Symbol.(adjustments))) + sum(Term.(Symbol.(adjustments .* "^2")))

FormulaTerm
Response:
  low_quality(unknown)
Predictors:
  Income(unknown)
  Gender(unknown)
  Age(unknown)
  Education(unknown)
  Income^2(unknown)
  Gender^2(unknown)
  Age^2(unknown)
  Education^2(unknown)

In [165]:
fit = glm(f, w6, Binomial(), ProbitLink())


ArgumentError: ArgumentError: There isn't a variable called 'Income^2' in your data; the nearest names appear to be: Income

In [151]:
fit = glm(Term(Symbol(treatment)) ~ sum(Term.(Symbol.(adjustments))), w6, Binomial(), ProbitLink())


StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, Matrix{Float64}}

low_quality ~ 1 + Income + Gender + Age + Education

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      z  Pr(>|z|)  Lower 95%    Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)  -1.98531    0.539008    -3.68    0.0002  -3.04175   -0.928876
Income        0.0974128  0.0705583    1.38    0.1674  -0.040879   0.235705
Gender        0.119665   0.232427     0.51    0.6067  -0.335884   0.575214
Age          -0.0141463  0.00962958  -1.47    0.1418  -0.03302    0.00472732
Education    -0.0697506  0.07664     -0.91    0.3628  -0.219962   0.0804611
────────────────────────────────────────────────────────────────────────────

In [156]:
describe(1 ./ (predict(fit, w6)))

Summary Stats:
Length:         1178
Missing Count:  0
Mean:           178.503841
Minimum:        15.207013
1st Quartile:   81.962500
Median:         122.873158
3rd Quartile:   211.657930
Maximum:        3152.840239
Type:           Float64


In [140]:
ip_weighting(w6, adjustments, treatment)

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, Matrix{Float64}}

low_quality ~ 1 + Income + Political_Views + Gender + Age + Education

Coefficients:
─────────────────────────────────────────────────────────────────────────────────
                      Coef.  Std. Error      z  Pr(>|z|)   Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────────────
(Intercept)      -1.84985    0.589574    -3.14    0.0017  -3.00539    -0.694301
Income            0.097154   0.0705293    1.38    0.1684  -0.0410809   0.235389
Political_Views  -0.036744   0.0755709   -0.49    0.6268  -0.18486     0.111372
Gender            0.0941508  0.235736     0.40    0.6896  -0.367883    0.556185
Age              -0.013548   0.00971554  -1.39    0.1632  -0.0325901   0.00549408
Education        -0.0729652  0.0766388   -0.95    0.3